# Analysis of exogenous plant status
Routines to evaluate the status of exogenously given power plants and storage units

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pommesevaluation.tools import update_matplotlib_params
from pommesevaluation.global_vars import (
    FUELS, 
    FUELS_RENAMED, 
    RES, 
    RES_TO_GROUP, 
    RES_RENAMED, 
    STORAGES, 
    STORAGES_RENAMED,
)

## Define data sets and global variables
Use the following data sets:
* maximum installed capacities for exogenously given power plants
* maximum power output for exogenously given plants per year &rarr; indicates installed power for that particular year
* capacities for exogenously given storage units
* maximum installed capacities for RES units
* RES generation time series

Output files comprise
* installed capacity for backup generation
* installed storage capacity
* installed VRES capacities
* annual VRES generation

In [ ]:
# Global settings
LANGUAGE = "German"
legend_loc = "bottom"  # "bottom", "side"

# inputs and outputs
path_folder = "./model_inputs/pommesinvest/"
file_name_transformers = "transformers_exogenous.csv"
file_name_transformers_max = "transformers_exogenous_max_ts.csv"
file_name_storages = "storages_el_exogenous.csv"
file_name_res_capacities = "sources_renewables_investment_model.csv"
file_name_res_generation = "sources_renewables_ts_hourly.csv"

path_data_out = "./data_out/"
path_plots = "./plots/"
output_file_names = {
    "backup": f"installed_capacity_backup_generation_{LANGUAGE}",
    "storages": f"installed_storage_capacity_{LANGUAGE}",
    "res_capacities": f"installed_capacity_res_{LANGUAGE}",
    "res_generation": f"annual_generation_res_{LANGUAGE}"
}

# Plotting and renaming
PLOT_LABELS = {
    "German": {
        "xlabel": "Jahr",
        "ylabel": {
            "power": "Installierte Leistung in MW",
            "energy": "Stromerzeugung in MWh/a",
        }
    },
    "English":  {
        "xlabel": "year",
        "ylabel": {
            "power": "installed power in MW",
            "energy": "power generation in MWh/a",
        }
    },
}
if legend_loc == "bottom":
    figsize=(18, 9)
else:
    figsize=(18, 7)

In [ ]:
update_matplotlib_params(
    small_size=18, medium_size=20, large_size=22
)

## Rearrange and plot data for transformers
* Combine time series and maximum installed capacities to obtain capacities per year
* Groupby fuel and year
* Draw and show plot

In [ ]:
transformers_capacities = pd.read_csv(f"{path_folder}{file_name_transformers}", index_col=0)["capacity"]
transformers_max_ts = pd.read_csv(f"{path_folder}{file_name_transformers_max}", index_col=0)

# Combine information to derive installed capacities by year
transformers_capacity_ts = transformers_max_ts.mul(transformers_capacities)

# Do some reformatting
transformers_capacity_ts.index = transformers_capacity_ts.index.str[:4]
transformers_capacity_ts_transposed = transformers_capacity_ts.T
transformers_capacity_ts_transposed.index = transformers_capacity_ts_transposed.index.str.split("_", expand=True)
transformers_capacity_ts_transposed.index.names = [
    "country", "component_name", "fuel", "type", "cluster", "cluster_no"
]
transformers_capacity_ts_transposed.reset_index(inplace=True)
transformers_capacity_ts_transposed = transformers_capacity_ts_transposed.loc[
    transformers_capacity_ts_transposed["country"] == "DE"
]

# groupby fuel
capacity_by_fuel_year =  transformers_capacity_ts_transposed.groupby("fuel").sum().T

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.set_axisbelow(True)
capacity_by_fuel_year = capacity_by_fuel_year[[col for col in FUELS.keys()]]
capacity_by_fuel_year.rename(columns=FUELS_RENAMED[LANGUAGE], inplace=True)
capacity_by_fuel_year.to_csv(f"{path_data_out}{output_file_names['backup']}.csv")
color = {FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS}
_ = capacity_by_fuel_year.loc["2020":"2045"].plot(kind="bar", stacked=True, ax=ax, color=color, edgecolor="#2A2A2A")
_ = plt.xlabel(PLOT_LABELS[LANGUAGE]["xlabel"], labelpad=10)
_ = plt.ylabel(PLOT_LABELS[LANGUAGE]["ylabel"]["power"], labelpad=10)
_ = ax.grid(axis="y", color="lightgrey")
if legend_loc == "bottom":
    _ = plt.legend(loc='upper center', bbox_to_anchor=[0.5, -0.25], fancybox=True, shadow=False, ncol=5)
else:
    _ = plt.legend(bbox_to_anchor=[1.3, 1.02], fancybox=True)
current_values = plt.gca().get_yticks()
if LANGUAGE == "English":
    _ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
elif LANGUAGE == "German":
    _ = plt.gca().set_yticklabels(['{:,.0f}'.format(x).replace(",", ".") for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['backup']}.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

## Rearrange and plot data for storages
* Read in data
* Reshape and repeat since installed values are kept constant

In [ ]:
storages = pd.read_csv(f"{path_folder}{file_name_storages}", index_col=0)
storages = storages.loc[
    storages["country"] == "DE", 
    ["capacity_pump", "capacity_turbine", "nominal_storable_energy"]
]
storages.rename(
    columns={"capacity_pump": "inflow_power", "capacity_turbine": "outflow_power"},
    inplace=True
)

In [ ]:
storages = pd.DataFrame(
    index=range(2020, 2051), 
    columns=storages["inflow_power"].index, 
    data={
        storages["inflow_power"].index[0]: storages["inflow_power"].values[0],
    }
)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
storages = storages[[f"DE_storage_el_{col}" for col in STORAGES if f"DE_storage_el_{col}" in storages.columns]]
storages.rename(columns={f"DE_storage_el_{k}": v for k, v in STORAGES_RENAMED[LANGUAGE].items()}, inplace=True)
storages.to_csv(f"{path_data_out}{output_file_names['storages']}.csv")
color = {STORAGES_RENAMED[LANGUAGE][f]: STORAGES[f] for f in STORAGES_RENAMED[LANGUAGE] if f in STORAGES}
_ = storages.loc["2020":"2045"].plot(kind="bar", stacked=True, ax=ax, color=color, edgecolor="#2A2A2A")
_ = plt.xlabel(PLOT_LABELS[LANGUAGE]["xlabel"], labelpad=10)
_ = plt.ylabel(PLOT_LABELS[LANGUAGE]["ylabel"]["power"], labelpad=10)
if legend_loc == "bottom":
    _ = plt.legend(loc='upper center', bbox_to_anchor=[0.5, -0.25], fancybox=True, shadow=False, ncol=3)
else:
    _ = plt.legend(bbox_to_anchor=[1.02, 1.02], fancybox=True)
current_values = plt.gca().get_yticks()
if LANGUAGE == "English":
    _ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
elif LANGUAGE == "German":
    _ = plt.gca().set_yticklabels(['{:,.0f}'.format(x).replace(",", ".") for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['storages']}.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

## Rearrange and plot data for renewables
* Read in installed capacities for 2020 and generation patterns
* Groupby year, calculate and plot annual installed capacities
* Groupby year, calculate and plot annual overall generation

In [ ]:
res_capacities_2020 = pd.read_csv(f"{path_folder}{file_name_res_capacities}", index_col=0)
res_generation = pd.read_csv(f"{path_folder}{file_name_res_generation}", index_col=0)
res_capacities_2020 = res_capacities_2020["capacity"]

In [ ]:
# Groupby year and use maximum and rescale 2020 to a maximum value of 1 to isolate installed capacities
grouped_res = res_generation.groupby(res_generation.index.str[:4])
grouped_res_max = grouped_res.max()
grouped_res_max = grouped_res_max.div(grouped_res_max.loc["2020"])
installed_res_capacities = grouped_res_max.mul(res_capacities_2020).round(0)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.set_axisbelow(True)
installed_res_capacities = installed_res_capacities[[col for col in RES_RENAMED[LANGUAGE] if col in installed_res_capacities.columns]]
for category, contents in RES_TO_GROUP.items():
    installed_res_capacities[category] = installed_res_capacities[contents].sum(axis=1)
    installed_res_capacities.drop(columns=contents, inplace=True)

installed_res_capacities.rename(
    columns={
        col: RES_RENAMED[LANGUAGE][col] 
        for col in RES_RENAMED[LANGUAGE] 
        if col in installed_res_capacities.columns
    }, 
    inplace=True,
)
installed_res_capacities.to_csv(f"{path_data_out}{output_file_names['res_capacities']}.csv")
color = {RES_RENAMED[LANGUAGE][f]: RES[f] for f in RES_RENAMED[LANGUAGE] if f in RES}
_ = installed_res_capacities.loc["2020":"2045"].plot(kind="bar", stacked=True, ax=ax, color=color, edgecolor="#2A2A2A")
_ = plt.xlabel(PLOT_LABELS[LANGUAGE]["xlabel"], labelpad=10)
_ = plt.ylabel(PLOT_LABELS[LANGUAGE]["ylabel"]["power"], labelpad=10)
_ = ax.grid(axis="y", color="lightgrey")
if legend_loc == "bottom":
    _ = plt.legend(loc='upper center', bbox_to_anchor=[0.5, -0.25], fancybox=True, shadow=False, ncol=3)
else:
    _ = plt.legend(bbox_to_anchor=[1.02, 1.02], fancybox=True)
current_values = plt.gca().get_yticks()
if LANGUAGE == "English":
    _ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
elif LANGUAGE == "German":
    _ = plt.gca().set_yticklabels(['{:,.0f}'.format(x).replace(",", ".") for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['res_capacities']}.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Use sum as aggregation rule for generation
grouped_res_sum = grouped_res.sum()
annual_res_generation_in_GWh = grouped_res_sum.mul(res_capacities_2020).round(0).div(1000)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
annual_res_generation_in_GWh = annual_res_generation_in_GWh[[col for col in RES_RENAMED[LANGUAGE] if col in annual_res_generation_in_GWh.columns]]
for category, contents in RES_TO_GROUP.items():
    annual_res_generation_in_GWh[category] = annual_res_generation_in_GWh[contents].sum(axis=1)
    annual_res_generation_in_GWh.drop(columns=contents, inplace=True)

annual_res_generation_in_GWh.rename(
    columns={
        col: RES_RENAMED[LANGUAGE][col] 
        for col in RES_RENAMED[LANGUAGE] 
        if col in annual_res_generation_in_GWh.columns
    }, 
    inplace=True,
)
annual_res_generation_in_GWh.to_csv(f"{path_data_out}{output_file_names['res_generation']}.csv")
color = {RES_RENAMED[LANGUAGE][f]: RES[f] for f in RES_RENAMED[LANGUAGE] if f in RES}
_ = annual_res_generation_in_GWh.loc["2020":"2045"].plot(kind="bar", stacked=True, ax=ax, color=color, edgecolor="#2A2A2A")
_ = plt.xlabel(PLOT_LABELS[LANGUAGE]["xlabel"], labelpad=10)
_ = plt.ylabel(PLOT_LABELS[LANGUAGE]["ylabel"]["energy"], labelpad=10)
if legend_loc == "bottom":
    _ = plt.legend(loc='upper center', bbox_to_anchor=[0.5, -0.25], fancybox=True, shadow=False, ncol=3)
else:
    _ = plt.legend(bbox_to_anchor=[1.02, 1.02], fancybox=True)
current_values = plt.gca().get_yticks()
if LANGUAGE == "English":
    _ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
elif LANGUAGE == "German":
    _ = plt.gca().set_yticklabels(['{:,.0f}'.format(x).replace(",", ".") for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['res_generation']}.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()